In [2]:
#!pip install python-dotenv
#!conda install -y -c conda-forge xgboost
#!dir -ls /tmp

total 17773692
       4 drwxr-xr-x 2 role-agent role-agent        4096 Jan 21 14:15 hsperfdata_role-agent
15566304 -rw-rw-r-- 1 ec2-user   ec2-user   15939866624 Jan 21 16:15 ImbDS3mill_train.csv
 2207380 -rw-rw-r-- 1 ec2-user   ec2-user    2260352613 Jan 21 15:35 ImbDS3mill_valid.csv
       4 drwxr-xr-x 3 role-agent role-agent        4096 Jan 21 14:15 jetty-0.0.0.0-9081-role-proxy-agent.war-_-any-3805171590929210667.dir


In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
# import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
import ftplib
from matplotlib import pyplot as plt
from collections import Counter
from dotenv import find_dotenv, load_dotenv
from scipy.stats.stats import pearsonr

import data_classes
import make_dataset as md
import build_data as bd
load_dotenv(find_dotenv())

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.set_option('io.hdf.default_format','table')

2019-01-21 21:47:43,971 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'XGBoost') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'XGBoost')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = np.float32
NP_FLOAT = np.dtype('float32')

train_dir = 'chuncks_random_c1millx2_train' #'chuncks_random_c1mill_slices' #'chuncks_random_c1millx2_train'
valid_dir = 'chuncks_random_c1millx2_valid'
test_dir = ''
train_period=[121,323] #[121,279] #[121, 143] 
valid_period=[324,329] #[280,285] #[144, 147] 
test_period=[330,342] #[286,304] #[148, 155]
batch_size=100000

print(RAW_DIR, PRO_DIR)

/home/ec2-user/SageMaker/XGBoost /home/ec2-user/SageMaker/XGBoost


In [3]:
import sagemaker
import boto3
from sagemaker.predictor import csv_serializer    # Converts strings for HTTP POST requests on inference
from time import gmtime, strftime                 

 
region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')

2019-01-21 21:47:50,240 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 21:47:50,245 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 21:47:50,246 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 21:47:50,248 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 21:47:50,249 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 21:47:50,250 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 21:47:50,251 - botocore.hooks - DEBUG - Changing e

In [4]:
print(region)
print(smclient)

us-east-2


In [5]:
from sagemaker import get_execution_role

role = sagemaker.get_execution_role()
print(role)

2019-01-21 21:47:56,601 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 21:47:56,606 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 21:47:56,607 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 21:47:56,610 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 21:47:56,610 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 21:47:56,611 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 21:47:56,613 - botocore.hooks - DEBUG - Changing e

arn:aws:iam::099048829003:role/service-role/AmazonSageMaker-ExecutionRole-20181010T181868


In [6]:
# ARN of the bucket:
#arn:aws:s3:::sagemaker-eduardo-zea
bucket = 'sagemaker-eduardo-zea'
prefix = 'sagemaker/mortgage/imbalanced-weighted'
print(bucket, prefix)

sagemaker-eduardo-zea sagemaker/mortgage/imbalanced-weighted


In [8]:
!dir -l /tmp/

total 11881588
drwxr-xr-x 2 role-agent role-agent       4096 Jan 21 14:15 hsperfdata_role-agent
-rw-rw-r-- 1 ec2-user   ec2-user   9906373885 Jan 21 21:01 ImbDS3mill_train.csv
-rw-rw-r-- 1 ec2-user   ec2-user   2260352613 Jan 21 21:34 ImbDS3mill_valid.csv
drwxr-xr-x 3 role-agent role-agent       4096 Jan 21 14:15 jetty-0.0.0.0-9081-role-proxy-agent.war-_-any-3805171590929210667.dir


In [9]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0.3",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "15",
          "MinValue": "5",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "20",
          "MinValue": "6",
          "Name": "max_depth"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_delta_step"
        }
          
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 10,
      "MaxParallelTrainingJobs": 1
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:merror",
      "Type": "Minimize"
    },   
    "TrainingJobEarlyStoppingType" : "Auto"
  } 

In [13]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(boto3.Session().region_name, 'xgboost')

s3_input_train = 's3://{}/{}/train'.format(bucket, prefix)
s3_input_validation ='s3://{}/{}/validation/'.format(bucket, prefix)

print(s3_input_train, s3_input_validation)
     
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated", #"ShardedByS3Key",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_train
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated", #"ShardedByS3Key",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_validation
          } 
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 1,
      "InstanceType": "ml.m5.2xlarge", #"ml.c4.2xlarge",
      "VolumeSizeInGB": 22 # the platform download the entire file for each 
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "eval_metric": "merror",  # "auc",
      "num_round": "15",
      "objective": "multi:softmax", #"rate_drop": "0.3",
      "tweedie_variance_power": "1.4",
      "num_class": "7",
      "csv_weights ": "1" # 1 for weighted instances
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 430000
    }
}
tuning_job_name = "XGBWgthImbMortgages002"

2019-01-21 23:01:48,081 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 23:01:48,086 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 23:01:48,087 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 23:01:48,090 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 23:01:48,091 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 23:01:48,092 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 23:01:48,093 - botocore.hooks - DEBUG - Changing e

s3://sagemaker-eduardo-zea/sagemaker/mortgage/imbalanced-weighted/train s3://sagemaker-eduardo-zea/sagemaker/mortgage/imbalanced-weighted/validation/


In [14]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                           HyperParameterTuningJobConfig = tuning_job_config,
                                           TrainingJobDefinition = training_job_definition)

2019-01-21 23:01:52,815 - botocore.hooks - DEBUG - Event before-parameter-build.sagemaker.CreateHyperParameterTuningJob: calling handler <function generate_idempotent_uuid at 0x7fa497b081e0>
2019-01-21 23:01:52,817 - botocore.endpoint - DEBUG - Making request for OperationModel(name=CreateHyperParameterTuningJob) with params: {'url_path': '/', 'query_string': '', 'method': 'POST', 'headers': {'X-Amz-Target': 'SageMaker.CreateHyperParameterTuningJob', 'Content-Type': 'application/x-amz-json-1.1', 'User-Agent': 'Boto3/1.9.69 Python/3.6.5 Linux/4.14.77-70.82.amzn1.x86_64 Botocore/1.12.69'}, 'body': b'{"HyperParameterTuningJobName": "XGBWgthImbMortgages002", "HyperParameterTuningJobConfig": {"ParameterRanges": {"CategoricalParameterRanges": [], "ContinuousParameterRanges": [{"MaxValue": "1", "MinValue": "0.3", "Name": "eta"}, {"MaxValue": "2", "MinValue": "0", "Name": "alpha"}, {"MaxValue": "15", "MinValue": "5", "Name": "min_child_weight"}], "IntegerParameterRanges": [{"MaxValue": "20", "

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-2:099048829003:hyper-parameter-tuning-job/xgbwgthimbmortgages002',
 'ResponseMetadata': {'RequestId': '60a98d0a-8674-49a8-8a2f-2fd9d799a68d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '60a98d0a-8674-49a8-8a2f-2fd9d799a68d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Mon, 21 Jan 2019 23:01:52 GMT'},
  'RetryAttempts': 0}}

In [17]:
tuner_parent_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
if not tuner_parent_metrics.dataframe().empty:
    df_parent = tuner_parent_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True)    
df_parent

2019-01-21 14:26:45,321 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2019-01-21 14:26:45,326 - botocore.hooks - DEBUG - Changing event name from before-call.apigateway to before-call.api-gateway
2019-01-21 14:26:45,327 - botocore.hooks - DEBUG - Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
2019-01-21 14:26:45,330 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
2019-01-21 14:26:45,331 - botocore.hooks - DEBUG - Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
2019-01-21 14:26:45,331 - botocore.hooks - DEBUG - Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
2019-01-21 14:26:45,333 - botocore.hooks - DEBUG - Changing e

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,alpha,eta,max_delta_step,max_depth,min_child_weight
4,0.493467,1980.0,2019-01-18 22:26:39+00:00,XGBmortgages013-006-63a079ba,Completed,2019-01-18 21:53:39+00:00,1.679463,0.308051,6.0,15.0,8.916770
5,0.494572,1993.0,2019-01-18 21:51:16+00:00,XGBmortgages013-005-b30684bf,Completed,2019-01-18 21:18:03+00:00,1.201588,0.300000,10.0,15.0,9.049803
1,0.496096,2001.0,2019-01-19 00:28:04+00:00,XGBmortgages013-009-d17c87d7,Completed,2019-01-18 23:54:43+00:00,1.505116,0.317685,8.0,15.0,8.406729
0,0.496476,1738.0,2019-01-19 01:05:34+00:00,XGBmortgages013-010-11c7604a,Completed,2019-01-19 00:36:36+00:00,1.755391,0.300000,5.0,13.0,5.732756
3,0.496496,2093.0,2019-01-18 23:04:03+00:00,XGBmortgages013-007-0280007f,Completed,2019-01-18 22:29:10+00:00,1.967702,0.418216,7.0,15.0,5.393729
2,0.497848,2225.0,2019-01-18 23:48:34+00:00,XGBmortgages013-008-621db045,Completed,2019-01-18 23:11:29+00:00,1.877675,0.492343,10.0,15.0,8.169249
8,0.498286,1480.0,2019-01-18 20:07:32+00:00,XGBmortgages013-002-53581e16,Completed,2019-01-18 19:42:52+00:00,1.811308,0.477480,10.0,11.0,6.292960
7,0.498362,1573.0,2019-01-18 20:38:47+00:00,XGBmortgages013-003-fce1916d,Completed,2019-01-18 20:12:34+00:00,1.249440,0.678499,10.0,12.0,9.247733
9,0.503714,1330.0,2019-01-18 19:39:26+00:00,XGBmortgages013-001-d474bb68,Completed,2019-01-18 19:17:16+00:00,1.174794,0.853793,7.0,10.0,9.797598
6,0.504076,1969.0,2019-01-18 21:15:06+00:00,XGBmortgages013-004-c542c3be,Completed,2019-01-18 20:42:17+00:00,1.074574,0.619064,4.0,15.0,5.920523


In [19]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent['FinalObjectiveValue'] > -float('inf')]

p = figure(plot_width=900, plot_height=400, x_axis_type='datetime',x_axis_label='datetime', y_axis_label='validation:merror')
p.circle(source=df_parent_objective_value, x='TrainingStartTime', y='FinalObjectiveValue', color='black')

show(p)

Loading BokehJS ...

In [24]:
#from sagemaker.tuner import WarmStartConfig, WarmStartTypes
#parent_tuning_job_name = tuning_job_name
#warm_start_config = WarmStartConfig(WarmStartTypes.IDENTICAL_DATA_AND_ALGORITHM, parents={parent_tuning_job_name})
#print(warm_start_config)

#Using boto3 API:
warm_start_config = {
          "ParentHyperParameterTuningJobs" : [ 
          {"HyperParameterTuningJobName" :'XGBmortgages012'}, 
          {"HyperParameterTuningJobName" :'XGBmortgages013'}
          ],
          "WarmStartType" : "IdenticalDataAndAlgorithm"    
}

In [25]:
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = 'XGBmortgages014',
   HyperParameterTuningJobConfig = tuning_job_config, # See notebook for tuning configuration
   TrainingJobDefinition = training_job_definition, # See notebook for job definition
   WarmStartConfig = warm_start_config)

2019-01-21 14:53:07,032 - botocore.hooks - DEBUG - Event before-parameter-build.sagemaker.CreateHyperParameterTuningJob: calling handler <function generate_idempotent_uuid at 0x7f169712c598>
2019-01-21 14:53:07,034 - botocore.endpoint - DEBUG - Making request for OperationModel(name=CreateHyperParameterTuningJob) with params: {'url_path': '/', 'query_string': '', 'method': 'POST', 'headers': {'X-Amz-Target': 'SageMaker.CreateHyperParameterTuningJob', 'Content-Type': 'application/x-amz-json-1.1', 'User-Agent': 'Boto3/1.9.69 Python/3.6.5 Linux/4.14.77-70.82.amzn1.x86_64 Botocore/1.12.69'}, 'body': b'{"HyperParameterTuningJobName": "XGBmortgages014", "HyperParameterTuningJobConfig": {"ParameterRanges": {"CategoricalParameterRanges": [], "ContinuousParameterRanges": [{"MaxValue": "1", "MinValue": "0.3", "Name": "eta"}, {"MaxValue": "2", "MinValue": "1", "Name": "alpha"}, {"MaxValue": "15", "MinValue": "5", "Name": "min_child_weight"}], "IntegerParameterRanges": [{"MaxValue": "30", "MinValu

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-2:099048829003:hyper-parameter-tuning-job/xgbmortgages014',
 'ResponseMetadata': {'RequestId': '71c6464b-33f5-4e02-8217-89bac400d114',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '71c6464b-33f5-4e02-8217-89bac400d114',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Mon, 21 Jan 2019 14:53:06 GMT'},
  'RetryAttempts': 0}}